![banner](/custom/banner.png)


# FSLeyes Jupyter notebook


## Help


Use the built-in `help` function to get help on anything in the shell
environment, whether it is a module, function, class, or object:

    help(load)



## Overlays


The `overlayList` is a list which contains all of the overlays that have been
loaded into FSLeyes.  You can load overlays by using the `load` function,
e.g.:

    std = load('MNI152_T1_2mm.nii.gz')


You can remove an overlay simply by removing it from the `overlayList`:

    del overlayList[0]


## Display settings


You can use the `displayCtx` to access the display settings for an
overlay. Display settings for an overlay are divided between two objects:

 - A `Display` object, which contains general display settings
 - A `DisplayOpts` object, which contains display settings specific to the
   overlay type


You can access these objects like so:

    overlay = overlayList[0]
    display = displayCtx.getDisplay(overlay)
    opts    = displayCtx.getOpts(   overlay)


Then adjusting the overlay display settings is easy:


    display.brightness = 75
    opts.cmap          = 'hot'


## FSL commands


Many FSL commands can be called through Python functions, e.g.:

    flirt('src.nii.gz', 'ref.nii.gz', applyxfm=True, init='src2ref.mat')


For commands which produce output images, you can have the outputs loaded
directly into FSLeyes by using the special `LOAD` symbol:

    struc = Image('struct.nii.gz')
    bet(struc, LOAD)


To submit a command as a cluster job, use the `submit` flag:

    fnirt('src.nii.gz', 'ref.nii.gz', submit=True)


## User interface


You can access the current views in FSLeyes through the ``frame`` object:

    ortho = frame.viewPanels[0]


You also can add/remove views through the ``frame``:


    frame.removeViewPanel(ortho)
    frame.addViewPanel(Scene3DPanel)


## Screenshots


To insert a screenshot of the currently displayed scene into the notebook, use
the `screenshot` function:

    ortho = frame.viewPanels[0]
    screenshot(ortho)


## Reproducing a scene


The `getState` function returns a string which describes the current FSLeyes
state. This string describes the current FSLeyes user interface (views,
controls, and toolbars) as well as all loaded overlays and their display
settings.


You can also use the `setState` function to reproduce a previously generated
FSLeyes state.


To make your notebook reproduce your FSLeyes setup, set up your user interface
and overlays how you want, and then call `getState` to get a description of
the current FSLeyes setup:


    getState()


Now copy the string that `getState` generates, and then at the top of your
notebook add a call to `setState`, passing in that string, e.g.:

    fsleyesState = 'OrthoPanel\nlayout2|name=OrthoPanel 1;caption=...'
    setState(fsleyesState)


## For more information

See the FSLeyes and `fslpy` API documentation:

 - https://open.win.ox.ac.uk/pages/fsl/fsleyes/fsleyes/apidoc/
 - https://open.win.ox.ac.uk/pages/fsl/fslpy/

In [6]:
# General purpose imports to handle paths, files etc
import os
import os.path as op
import glob
import pandas as pd
import numpy as np
import json


# Useful functions to define and import datasets from open neuro
import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt



def reset_overlays():
    """
    Clears view and completely remove visualization. All files opened in FSLeyes are closed.
    The view (along with any color map) is reset to the regular ortho panel.
    """
    l = frame.overlayList
    while(len(l)>0):
        del l[0]
    frame.removeViewPanel(frame.viewPanels[0])
    # Put back an ortho panel in our viz for future displays
    frame.addViewPanel(OrthoPanel)
    
def mkdir_no_exist(path):
    if not op.isdir(path):
        os.makedirs(path)
        
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)


def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def direct_file_download_open_neuro(file_list, file_types, dataset_id, dataset_version, save_dirs):
    # https://openneuro.org/crn/datasets/ds004226/snapshots/1.0.0/files/sub-001:sub-001_scans.tsv
    for i, n in enumerate(file_list):
        subject = n.split('_')[0]
        download_link = 'https://openneuro.org/crn/datasets/{}/snapshots/{}/files/{}:{}:{}'.format(dataset_id, dataset_version, subject, file_types[i],n)
        print('Attempting download from ', download_link)
        download_url(download_link, op.join(save_dirs[i], n))
        print('Ok')
        
def get_json_from_file(fname):
    f = open(fname)
    data = json.load(f)
    f.close()
    return data

In [13]:
epi1 = 'sub-06_task-flanker_run-1_bold.nii.gz'
epi2 = 'sub-06_task-flanker_run-2_bold.nii.gz'


In [9]:
dataset_id = 'ds000102'
subject = '06' 

# Download one subject's data from each dataset
sample_path = "dataset"
mkdir_no_exist(sample_path)
bids_root = op.join(os.path.abspath(""),sample_path, dataset_id)
deriv_root = op.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')

mkdir_no_exist(bids_root)

In [11]:
print_dir_tree(bids_root, max_depth=4)

|ds000102/
|--- CHANGES
|--- README
|--- dataset_description.json
|--- participants.tsv
|--- sub-06/
|------ anat/
|--------- sub-06_T1w.nii.gz
|------ func/
|--------- sub-06_task-flanker_run-1_bold.nii.gz
|--------- sub-06_task-flanker_run-2_bold.nii.gz


In [10]:
os.system("""openneuro-py download --dataset {} --include sub-{}/anat/* 
          --include sub-{}/func/sub-{}_task-flanker_run-1_bold.nii.gz 
          --include sub-{}/func/sub-{}_task-flanker_run-2_bold.nii.gz 
          --target_dir {}""".format(dataset_id, subject, subject, subject, subject, subject, bids_root).replace("\n", " "))

0

# Part.1 Visualisation 

## Anatomical 

In [15]:
reset_overlays()
load(op.join(bids_root, 'sub-06', 'anat', 'sub-06_T1w.nii.gz'))

Image(sub-06_T1w, /home/francio/Desktop/MyFiles/NSSP_Mini1/dataset/ds000102/sub-06/anat/sub-06_T1w.nii.gz)

## Functional


In [18]:
reset_overlays()
load(op.join(bids_root, 'sub-06', 'func', epi1))

Image(sub-06_task-flanker_run-2_bold, /home/francio/Desktop/MyFiles/NSSP_Mini1/dataset/ds000102/sub-06/func/sub-06_task-flanker_run-2_bold.nii.gz)

## Brain extraction

In [34]:
anatomical_path = op.join(bids_root, 'sub-06', 'anat', 'sub-06_T1w.nii.gz')
betted_brain_path = op.join(preproc_root, 'sub-06', 'anat', 'sub-06_T1w')
resulting_mask_path = op.join(preproc_root, 'sub-06', 'anat', 'sub-06_T1w_mask')
#bet(anatomical_path, betted_brain_path, mask=resulting_mask_path)

In [23]:
reset_overlays()
load(op.join(bids_root, 'sub-06', 'anat', 'sub-06_T1w.nii.gz'))
load(resulting_mask_path)

Image(sub-06_T1w_mask, /home/francio/Desktop/MyFiles/NSSP_Mini1/dataset/ds000102/derivatives/preprocessed_data/sub-06/anat/sub-06_T1w_mask.nii.gz)

In [36]:
for fit in [fit * 0.1 for fit in range(0, 10)]:
    betted_brain_path = op.join(preproc_root, 'sub-06', 'anat', f'sub-06_T1w{fit}')
    resulting_mask_path = op.join(preproc_root, 'sub-06', 'anat', f'sub-06_T1w_mask{fit}')
    bet(anatomical_path, betted_brain_path, mask=resulting_mask_path,f=fit)

essai avec fit = 0.0
essai avec fit = 0.1
essai avec fit = 0.2
essai avec fit = 0.30000000000000004
essai avec fit = 0.4
essai avec fit = 0.5
essai avec fit = 0.6000000000000001
essai avec fit = 0.7000000000000001
essai avec fit = 0.8
essai avec fit = 0.9


{}

In [30]:
reset_overlays()
load(op.join(bids_root, 'sub-06', 'anat', 'sub-06_T1w.nii.gz'))
load(resulting_mask_path)

Image(sub-06_T1w_mask, /home/francio/Desktop/MyFiles/NSSP_Mini1/dataset/ds000102/derivatives/preprocessed_data/sub-06/anat/sub-06_T1w_mask.nii.gz)